In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
data=pd.read_csv("AKBNK-OpenToClose-features.csv",index_col='date', parse_dates=True)
data.head()

,volume,XU030,USDTRY,XBANK,SPX,NASDAQ,BistTem,BistTem25,BOVESPA,DAX,...,rvol,rvol_dolar,rvol_bist,return,return1,return2,return3,return4,return5,tomorrow
date,,,,,,,,,,,,,,,,,,,,,
2017-01-02,158.881926,-1.630061,1.375584,-1.666705,0.579978,0.253149,-1.459304,-1.567789,-1.065961,-0.121557,...,33.708315,22.770189,22.33834,-0.258355,-0.258355,-0.258355,-0.258355,-0.258355,-0.258355,-2.209041
2017-01-03,158.881926,-1.630061,1.375584,-1.666705,0.579978,0.253149,-1.459304,-1.567789,-1.065961,-0.121557,...,33.708315,22.770189,22.33834,-2.209041,-0.258355,-0.258355,-0.258355,-0.258355,-0.258355,-1.717475
2017-01-04,-23.655408,-0.749518,-0.602815,-1.117738,0.579978,-0.008499,-0.950929,-1.246611,3.666117,0.000604,...,33.708315,22.770189,22.33834,-1.717475,-2.209041,-0.258355,-0.258355,-0.258355,-0.258355,-0.796940
2017-01-05,-9.223885,0.439424,0.552719,-0.157964,-0.062122,-0.010625,0.200238,0.621157,-0.364287,0.005438,...,33.708315,22.770189,22.33834,-0.796940,-1.717475,-2.209041,-0.258355,-0.258355,-0.258355,1.980567
2017-01-06,-7.505213,0.897596,1.524976,1.335595,0.358100,0.057359,0.946680,0.802795,0.779431,0.121377,...,33.708315,22.770189,22.33834,1.980567,-0.796940,-1.717475,-2.209041,-0.258355,-0.258355,0.786504


In [ ]:
# "tomorrow" değişkenini dönüştürme
data['tomorrow'] = data['tomorrow'].apply(lambda x: 1 if x > 0 else 0)

tomorrow_values = data["tomorrow"]  # Access the "tomorrow" row
count_ones = np.count_nonzero(tomorrow_values == 1)  # Count the occurrences of 1
count_zeros = np.count_nonzero(tomorrow_values == 0)  # Count the occurrences of 0

print("Number of ones:", count_ones)
print("Number of zeros:", count_zeros)

data1=data.drop("code",axis=1)

Number of ones: 686
Number of zeros: 903


**XGBoost & RFE**

In [ ]:
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE

window_size = 20
num_features_to_select = 20
selected_features = []

for i in range(0, len(data1) - window_size + 1, window_size):
    subset = data1.iloc[i:i+window_size]
    X = subset.drop("tomorrow", axis=1)
    y = subset["tomorrow"]

    model = xgb.XGBClassifier(eval_metric='error')
    rfe = RFE(estimator=model, n_features_to_select=num_features_to_select)
    X_rfe = rfe.fit_transform(X, y)

    selected_features_indexes = rfe.get_support(indices=True)
    selected_features_names = X.columns[selected_features_indexes]
    selected_features_importance = rfe.estimator_.feature_importances_

    selected_features.append(list(zip(selected_features_names, selected_features_importance)))

    print(f"{i//window_size + 1}. {window_size}-günlük dönemde en önemli parametreler:")
    sorted_features = sorted(selected_features[-1], key=lambda x: x[1], reverse=True)
    for rank, (feature, importance) in enumerate(sorted_features):
        print(f"{rank+1}. {feature} - Feature Importance: {importance}")
    print()


1. 20-günlük dönemde en önemli parametreler:
1. volume - Feature Importance: 0.2102278769016266
2. futureRatio4 - Feature Importance: 0.137086883187294
3. XU030 - Feature Importance: 0.10184740275144577
4. SENSEX - Feature Importance: 0.09031256288290024
5. sma3 - Feature Importance: 0.08137425035238266
6. Singapur - Feature Importance: 0.07138369977474213
7. MSCIEm-Ema - Feature Importance: 0.06168649345636368
8. cci5 - Feature Importance: 0.0599956251680851
9. Shanghai - Feature Importance: 0.0547182634472847
10. ema3 - Feature Importance: 0.044511161744594574
11. HALKB_volume - Feature Importance: 0.03398623317480087
12. SPX - Feature Importance: 0.021673962473869324
13. return2 - Feature Importance: 0.017616823315620422
14. i2 - Feature Importance: 0.013578670099377632
15. NASDAQ - Feature Importance: 0.0
16. BistTem - Feature Importance: 0.0
17. BistTem25 - Feature Importance: 0.0
18. mom5 - Feature Importance: 0.0
19. rsi14 - Feature Importance: 0.0
20. return1 - Feature Importan

In [ ]:
import pandas as pd

dfs = []  # DataFrame'leri tutacak liste

for j in range(0, 79):
    columns = []
    feature_importances = []

    for i in range(0, 20):
        columns.append(selected_features[j][i][0])
        feature_importances.append(selected_features[j][i][1])

    df = pd.DataFrame([feature_importances], columns=columns)
    dfs.append(df)

# DataFrame'leri birleştirme
df_combined = pd.concat(dfs, ignore_index=True)


In [ ]:
df_combined=df_combined.fillna(0.0)

In [ ]:
df_combined.index=np.arange(1,80)

In [ ]:
df_combined

,volume,XU030,SPX,NASDAQ,BistTem,BistTem25,MSCIEm-Ema,SENSEX,Shanghai,Singapur,...,TKImportCurn,YKBNK_PD_DD,12MTH_FORWARD_CPS,Süpermarket - Sentiment,TTM_net_asset_value,interest_div_income,net_interest_income,12MTH_FORWARD_ROE,BNP,Noninterest_Income_over_Op_Inc
1,0.210228,0.101847,0.021674,0.0000,0.000000,0.000000,0.061686,0.090313,0.054718,0.071384,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.023402,0.000000,0.051784,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.000000,0.008565,0.000000,0.0000,0.000000,0.000000,0.000000,0.113937,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.004043,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.000000,0.000000,0.000000,0.0000,0.035145,0.000000,0.000000,0.007949,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,0.071504,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
76,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
77,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.041304
78,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.013527,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
from sklearn.cluster import KMeans

n_clusters = 5 # Küme sayısı

kmeans = KMeans(n_clusters=n_clusters)
kmeans.fit(df_combined)

# Küme etiketlerini almak
labels = kmeans.labels_

# Sonuçları yazdırmak
results = pd.DataFrame({'Satır': range(1, len(df_combined) + 1), 'Küme': labels})
#print(results)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
print(len(results[results["Küme"]==4]))
print(len(results[results["Küme"]==3]))
print(len(results[results["Küme"]==2]))
print(len(results[results["Küme"]==1]))
print(len(results[results["Küme"]==0]))

9
9
15
15
31


In [ ]:
index_list_cluster4 = results[results["Küme"] == 4].index.tolist()
index_list_cluster3 = results[results["Küme"] == 3].index.tolist()
index_list_cluster2 = results[results["Küme"] == 2].index.tolist()
index_list_cluster1 = results[results["Küme"] == 1].index.tolist()
index_list_cluster0 = results[results["Küme"] == 0].index.tolist()


# Boş bir liste oluşturma
chunks = []

# Her bir indeks için parçaları birleştirme
for i in index_list_cluster4:
    chunk = data1[((i-1)*20) : ((i)*20)]
    chunks.append(chunk)

# DataFrame'leri birleştirme
df_combined4 = pd.concat(chunks)

df_combined4


# Boş bir liste oluşturma
chunks = []

# Her bir indeks için parçaları birleştirme
for i in index_list_cluster3:
    chunk = data1[((i-1)*20) : ((i)*20)]
    chunks.append(chunk)

# DataFrame'leri birleştirme
df_combined3 = pd.concat(chunks)

df_combined3

# Boş bir liste oluşturma
chunks = []

# Her bir indeks için parçaları birleştirme
for i in index_list_cluster2:
    chunk = data1[((i-1)*20) : ((i)*20)]
    chunks.append(chunk)

# DataFrame'leri birleştirme
df_combined2 = pd.concat(chunks)

df_combined2

# Boş bir liste oluşturma
chunks = []

# Her bir indeks için parçaları birleştirme
for i in index_list_cluster1:
    chunk = data1[((i-1)*20) : ((i)*20)]
    chunks.append(chunk)

# DataFrame'leri birleştirme
df_combined1 = pd.concat(chunks)

df_combined1
# Boş bir liste oluşturma
chunks = []

# Her bir indeks için parçaları birleştirme
for i in index_list_cluster0:
    chunk = data1[((i-1)*20) : ((i)*20)]
    chunks.append(chunk)

# DataFrame'leri birleştirme
df_combined0 = pd.concat(chunks)

df_combined0

,volume,XU030,USDTRY,XBANK,SPX,NASDAQ,BistTem,BistTem25,BOVESPA,DAX,...,rvol,rvol_dolar,rvol_bist,return,return1,return2,return3,return4,return5,tomorrow
date,,,,,,,,,,,,,,,,,,,,,
2017-01-02,158.881926,-1.630061,1.375584,-1.666705,0.579978,0.253149,-1.459304,-1.567789,-1.065961,-0.121557,...,33.708315,22.770189,22.338340,-0.258355,-0.258355,-0.258355,-0.258355,-0.258355,-0.258355,0
2017-01-03,158.881926,-1.630061,1.375584,-1.666705,0.579978,0.253149,-1.459304,-1.567789,-1.065961,-0.121557,...,33.708315,22.770189,22.338340,-2.209041,-0.258355,-0.258355,-0.258355,-0.258355,-0.258355,0
2017-01-04,-23.655408,-0.749518,-0.602815,-1.117738,0.579978,-0.008499,-0.950929,-1.246611,3.666117,0.000604,...,33.708315,22.770189,22.338340,-1.717475,-2.209041,-0.258355,-0.258355,-0.258355,-0.258355,0
2017-01-05,-9.223885,0.439424,0.552719,-0.157964,-0.062122,-0.010625,0.200238,0.621157,-0.364287,0.005438,...,33.708315,22.770189,22.338340,-0.796940,-1.717475,-2.209041,-0.258355,-0.258355,-0.258355,1
2017-01-06,-7.505213,0.897596,1.524976,1.335595,0.358100,0.057359,0.946680,0.802795,0.779431,0.121377,...,33.708315,22.770189,22.338340,1.980567,-0.796940,-1.717475,-2.209041,-0.258355,-0.258355,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-15,-10.579122,2.467273,0.096049,7.046549,-2.756768,-3.346462,1.718987,1.971082,-0.365374,-3.332677,...,53.130777,1.053864,36.029753,3.801100,-1.435504,0.632028,3.940202,2.552179,3.000394,0
2022-12-16,-10.439225,0.216618,0.052011,2.009114,-0.847512,-1.037563,0.673646,0.737524,-0.485045,-0.668312,...,47.436826,0.905399,33.871950,-0.053768,3.801100,-1.435504,0.632028,3.940202,2.552179,1
2022-12-19,17.938965,3.900767,-0.024125,9.372642,-1.184556,-1.350155,2.193750,2.154621,1.605842,0.357811,...,55.597373,0.747624,36.467798,7.756751,-0.053768,3.801100,-1.435504,0.632028,3.940202,0


# 0 Cluster'ı için ÖZELLİK SEÇİMİ VE XGBOOST SINIFLANDIRMA

In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier



# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
X = df_combined0.drop('tomorrow', axis=1)
y = df_combined0['tomorrow']

# RFE modelini oluştur
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=10)

# RFE modelini veriye uygula ve seçilen özellikleri al
X_rfe = rfe.fit_transform(X, y)
seçilen_özellikler = X.columns[rfe.support_]

# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# XGBoost modelini veriyle eğit
xgb_model.fit(X_rfe, y)

# Seçilen özellikleri bastır
print("Seçilen Özellikler:")
for özellik in seçilen_özellikler:
    print(özellik)


Seçilen Özellikler:
BOVESPA
SENSEX
SET
HALKB_close
HALKB_volume
Deposit_Growth
IndProdMonthly
GARAN_FK
futureRatio1
futureRatio5


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

train_data=df_combined0[:-20]
test_data=df_combined0.tail(20)


train_data_x = train_data[seçilen_özellikler]
train_data_y = train_data["tomorrow"]

test_data_x = test_data[seçilen_özellikler]
test_data_y = test_data["tomorrow"]


# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
#X = df_combined0[seçilen_özellikler]
#y = df_combined0["tomorrow"]

# Veriyi eğitim ve test alt kümelerine ayır
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# GridSearchCV için hiperparametrelerin aday değerlerini belirle
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

# GridSearchCV modelini oluştur
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')

# GridSearchCV modelini eğit
grid_search.fit(train_data_x, train_data_y)


# En iyi modeli al
en_iyi_model = grid_search.best_estimator_

# Test veri kümesini kullanarak tahmin yap
tahminler_test = en_iyi_model.predict(test_data_x)
tahminler_train = en_iyi_model.predict(train_data_x)

# Tahminleri ve gerçek hedef değerleri karşılaştır
dogruluk_test = accuracy_score(test_data_y, tahminler_test)
dogruluk_train = accuracy_score(train_data_y, tahminler_train)

print('Test Doğruluk:', dogruluk_test)
print('Train Doğruluk:', dogruluk_train)


Test Doğruluk: 0.6
Train Doğruluk: 0.785


# 1 CLUSTER'I İÇİN ÖZELLİK SEÇİMİ VE XGBOOST

In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier



# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
X = df_combined1.drop('tomorrow', axis=1)
y = df_combined1['tomorrow']

# RFE modelini oluştur
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=10)

# RFE modelini veriye uygula ve seçilen özellikleri al
X_rfe = rfe.fit_transform(X, y)
seçilen_özellikler = X.columns[rfe.support_]

# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# XGBoost modelini veriyle eğit
xgb_model.fit(X_rfe, y)

# Seçilen özellikleri bastır
print("Seçilen Özellikler:")
for özellik in seçilen_özellikler:
    print(özellik)


Seçilen Özellikler:
XU030
KLCI
Singapur
HALKB_close
HALKB_volume
VAKBN_volume
Citi-A
Dividend_Yield
Petrokimya - Sentiment
mom10


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

train_data=df_combined1[:-20]
test_data=df_combined1.tail(20)


train_data_x = train_data[seçilen_özellikler]
train_data_y = train_data["tomorrow"]

test_data_x = test_data[seçilen_özellikler]
test_data_y = test_data["tomorrow"]


# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
#X = df_combined0[seçilen_özellikler]
#y = df_combined0["tomorrow"]

# Veriyi eğitim ve test alt kümelerine ayır
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# GridSearchCV için hiperparametrelerin aday değerlerini belirle
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

# GridSearchCV modelini oluştur
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')

# GridSearchCV modelini eğit
grid_search.fit(train_data_x, train_data_y)


# En iyi modeli al
en_iyi_model = grid_search.best_estimator_

# Test veri kümesini kullanarak tahmin yap
tahminler_test = en_iyi_model.predict(test_data_x)
tahminler_train = en_iyi_model.predict(train_data_x)

# Tahminleri ve gerçek hedef değerleri karşılaştır
dogruluk_test = accuracy_score(test_data_y, tahminler_test)
dogruluk_train = accuracy_score(train_data_y, tahminler_train)

print('Test Doğruluk:', dogruluk_test)
print('Train Doğruluk:', dogruluk_train)


Test Doğruluk: 0.7
Train Doğruluk: 0.8428571428571429


# 2 CLUSTER'I İÇİN ÖZELLİK SEÇİMİ VE XGBOOST

In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier



# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
X = df_combined2.drop('tomorrow', axis=1)
y = df_combined2['tomorrow']

# RFE modelini oluştur
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=10)

# RFE modelini veriye uygula ve seçilen özellikleri al
X_rfe = rfe.fit_transform(X, y)
seçilen_özellikler = X.columns[rfe.support_]

# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# XGBoost modelini veriyle eğit
xgb_model.fit(X_rfe, y)

# Seçilen özellikleri bastır
print("Seçilen Özellikler:")
for özellik in seçilen_özellikler:
    print(özellik)


Seçilen Özellikler:
volume
USDTRY
FTSE
YKBNK_volume
Yatirim-A
roa
XU030/DAX
XU030/BOVESPA
sma4
return3


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

train_data=df_combined2[:-20]
test_data=df_combined2.tail(20)


train_data_x = train_data[seçilen_özellikler]
train_data_y = train_data["tomorrow"]

test_data_x = test_data[seçilen_özellikler]
test_data_y = test_data["tomorrow"]


# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
#X = df_combined0[seçilen_özellikler]
#y = df_combined0["tomorrow"]

# Veriyi eğitim ve test alt kümelerine ayır
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# GridSearchCV için hiperparametrelerin aday değerlerini belirle
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

# GridSearchCV modelini oluştur
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')

# GridSearchCV modelini eğit
grid_search.fit(train_data_x, train_data_y)


# En iyi modeli al
en_iyi_model = grid_search.best_estimator_

# Test veri kümesini kullanarak tahmin yap
tahminler_test = en_iyi_model.predict(test_data_x)
tahminler_train = en_iyi_model.predict(train_data_x)

# Tahminleri ve gerçek hedef değerleri karşılaştır
dogruluk_test = accuracy_score(test_data_y, tahminler_test)
dogruluk_train = accuracy_score(train_data_y, tahminler_train)

print('Test Doğruluk:', dogruluk_test)
print('Train Doğruluk:', dogruluk_train)


Test Doğruluk: 0.85
Train Doğruluk: 1.0


# 3 CLUSTER'I İÇİN ÖZELLİK SEÇİMİ VE XGBOOST

In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier



# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
X = df_combined3.drop('tomorrow', axis=1)
y = df_combined3['tomorrow']

# RFE modelini oluştur
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=14)

# RFE modelini veriye uygula ve seçilen özellikleri al
X_rfe = rfe.fit_transform(X, y)
seçilen_özellikler = X.columns[rfe.support_]

# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# XGBoost modelini veriyle eğit
xgb_model.fit(X_rfe, y)

# Seçilen özellikleri bastır
print("Seçilen Özellikler:")
for özellik in seçilen_özellikler:
    print(özellik)


Seçilen Özellikler:
SPX
DAX
EURTRY
MSCIEm
SET
GARAN_close
VAKBN_close
futureRatio
_Fee_Revenue
CAR_Tier2
futureRatio4
stoch_d
ema50
return3


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

train_data=df_combined3[:-20]
test_data=df_combined3.tail(20)


train_data_x = train_data[seçilen_özellikler]
train_data_y = train_data["tomorrow"]

test_data_x = test_data[seçilen_özellikler]
test_data_y = test_data["tomorrow"]


# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
#X = df_combined0[seçilen_özellikler]
#y = df_combined0["tomorrow"]

# Veriyi eğitim ve test alt kümelerine ayır
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# GridSearchCV için hiperparametrelerin aday değerlerini belirle
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300]
}

# GridSearchCV modelini oluştur
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')

# GridSearchCV modelini eğit
grid_search.fit(train_data_x, train_data_y)


# En iyi modeli al
en_iyi_model = grid_search.best_estimator_

# Test veri kümesini kullanarak tahmin yap
tahminler_test = en_iyi_model.predict(test_data_x)
tahminler_train = en_iyi_model.predict(train_data_x)

# Tahminleri ve gerçek hedef değerleri karşılaştır
dogruluk_test = accuracy_score(test_data_y, tahminler_test)
dogruluk_train = accuracy_score(train_data_y, tahminler_train)

print('Test Doğruluk:', dogruluk_test)
print('Train Doğruluk:', dogruluk_train)


Test Doğruluk: 0.6
Train Doğruluk: 0.99375


# 4 CLUSTER'I İÇİN ÖZELLİK SEÇİMİ VE XGBOOST



In [ ]:
import pandas as pd
from sklearn.feature_selection import RFE
from xgboost import XGBClassifier



# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
X = df_combined4.drop('tomorrow', axis=1)
y = df_combined4['tomorrow']

# RFE modelini oluştur
rfe = RFE(estimator=XGBClassifier(), n_features_to_select=6)

# RFE modelini veriye uygula ve seçilen özellikleri al
X_rfe = rfe.fit_transform(X, y)
seçilen_özellikler = X.columns[rfe.support_]

# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# XGBoost modelini veriyle eğit
xgb_model.fit(X_rfe, y)

# Seçilen özellikleri bastır
print("Seçilen Özellikler:")
for özellik in seçilen_özellikler:
    print(özellik)


Seçilen Özellikler:
XBANK
HALKB_volume
XU030/KOSPI
ema3
ema5
return2


In [ ]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

train_data=df_combined4[:-20]
test_data=df_combined4.tail(20)


train_data_x = train_data[seçilen_özellikler]
train_data_y = train_data["tomorrow"]

test_data_x = test_data[seçilen_özellikler]
test_data_y = test_data["tomorrow"]


# Özellikler (X) ve hedef değişken (y) olarak veriyi ayır
#X = df_combined0[seçilen_özellikler]
#y = df_combined0["tomorrow"]

# Veriyi eğitim ve test alt kümelerine ayır
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# XGBoost modelini oluştur
xgb_model = XGBClassifier()

# GridSearchCV için hiperparametrelerin aday değerlerini belirle
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [100, 200, 300],
}

# GridSearchCV modelini oluştur
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, scoring='accuracy')

# GridSearchCV modelini eğit
grid_search.fit(train_data_x, train_data_y)


# En iyi modeli al
en_iyi_model = grid_search.best_estimator_

# Test veri kümesini kullanarak tahmin yap
tahminler_test = en_iyi_model.predict(test_data_x)
tahminler_train = en_iyi_model.predict(train_data_x)

# Tahminleri ve gerçek hedef değerleri karşılaştır
dogruluk_test = accuracy_score(test_data_y, tahminler_test)
dogruluk_train = accuracy_score(train_data_y, tahminler_train)

print('Test Doğruluk:', dogruluk_test)
print('Train Doğruluk:', dogruluk_train)


Test Doğruluk: 0.75
Train Doğruluk: 0.825
